In [ ]:
%matplotlib  inline

import matplotlib.pyplot as pyplot
import matplotlib.patches as mpatches
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy
import pandas
import geopandas
import csv
import shapely.geometry
import mapclassify
import ipywidgets

pandas.options.display.max_rows = 24

In [ ]:
# Extract the polygon of mainland Portugal from the map shapefile, dropping the islands.
whole_portugal = geopandas.read_file('portugal_map_shapefile/DATA/Countries/PT/NUTS_3.shp', index=False)
islands = portugal_geo[(portugal_geo['NUTS_LABEL'] == 'Região Autónoma dos Açores') | (portugal_geo['NUTS_LABEL'] == 'Região Autónoma da Madeira')].index
mainland_portugal = portugal_geo.drop(index=islands)
portugal = mainland_portugal.to_crs(epsg=3763)

# Testing whether all the schools and restaurants coordinates are within mainland Portugal
portugal_polygon = portugal.unary_union

In [ ]:
# Extractig the polygons of the denser metropolitan areas
urbanas_geo = geopandas.read_file('portugal_map_shapefile/DATA/Countries/PT/BuiltupA.shp', index=False)
urbanas = urbanas_geo.to_crs(epsg=3763)
urbanas = urbanas[urbanas['geometry'].intersects(portugal_poligono) == True] #excluir as regioes autonomas
urbanas = urbanas[['geometry']]
urbanas_poligono = urbanas.unary_union
# Nota: como parte dos poligonos das urbanas ultrapassam ligeiramente o bordo do poligono portugal,
#tenho que usar .intersects e não .within

In [ ]:
vilas_geo = geopandas.read_file('portugal_map_shapefile/DATA/Countries/PT/BuiltupP.shp', index=False)
vilas = vilas_geo.to_crs(epsg=3763)
vilas = vilas[vilas['geometry'].within(portugal_poligono) == True] #excluir as regioes autonomas
vilas = vilas[['NAMA1','geometry']].rename({'NAMA1':'Nome'}, axis =1) #manter apenas info relevante

In [ ]:
escolas_original = pandas.read_csv('points_of_interest/schools.csv', sep = ';', index_col=False)
geometry_escolas = [Point(xy) for xy in zip(escolas_original['Lon'], escolas_original['Lat'])]
escolas_geo = geopandas.GeoDataFrame(escolas_original, geometry=geometry_escolas, crs='EPSG:4258') #confirmar epsg dce origem
escolas_crs = escolas_geo.to_crs(epsg=3763)
escolas_crs[escolas_crs['geometry'].within(portugal_poligono) == False] #testar coordenadas erradas

In [ ]:
mcd_original = pandas.read_csv('points_of_interest/mcdonalds.csv', sep = ';', index_col=False)
geometry_mcd = [Point(xy) for xy in zip(mcd_original['Lon'], mcd_original['Lat'])]
mcd_geo = geopandas.GeoDataFrame(mcd_original, geometry=geometry_mcd, crs='EPSG:4258') #nao deveria ser 3857? retirado do gmaps, right?
mcd_crs = mcd_geo.to_crs(epsg=3763)
mcd_crs[mcd_crs['geometry'].within(portugal_poligono) == False]

In [ ]:
tel_original = pandas.read_csv('points_of_interest/telepizza.csv', sep = ';', index_col=False)
geometry_tel = [Point(xy) for xy in zip(tel_original['Lon'], tel_original['Lat'])]
tel_geo = geopandas.GeoDataFrame(tel_original, geometry=geometry_tel, crs='EPSG:4258') #nao deveria ser 3857? retirado do gmaps, right?
tel_crs = tel_geo.to_crs(epsg=3763)
tel_crs[tel_crs['geometry'].within(portugal_poligono) == False]

In [ ]:
bk_original = pandas.read_csv('points_of_interest/burger_king.csv', sep = ';', index_col=False)
geometry_bk = [Point(xy) for xy in zip(bk_original['Lon'], bk_original['Lat'])]
bk_geo = geopandas.GeoDataFrame(bk_original, geometry=geometry_bk, crs='EPSG:4258') #nao deveria ser 3857? retirado do applemaps, right?
bk_crs = bk_geo.to_crs(epsg=3763)
bk_crs[bk_crs['geometry'].within(portugal_poligono) == False]

In [ ]:
#acrescentar NUT correspondente na base de dados das escolas e dos restaurantes
escolas = geopandas.sjoin(portugal, escolas_crs, predicate='contains', how='right')[['Nome', 'Lat', 'Lon', 'NUTS_LABEL', 'geometry']].sort_values(by='NUTS_LABEL').reset_index(drop=True)
bk = geopandas.sjoin(portugal, bk_crs, predicate='contains', how='right')[['Tipo','Nome', 'Lat', 'Lon','NUTS_LABEL', 'geometry']].sort_values(by='NUTS_LABEL').reset_index(drop=True)
mcd = geopandas.sjoin(portugal, mcd_crs, predicate='contains', how='right')[['Tipo','Nome', 'Lat', 'Lon','NUTS_LABEL', 'geometry']].sort_values(by='NUTS_LABEL').reset_index(drop=True)
tel = geopandas.sjoin(portugal, tel_crs, predicate='contains', how='right')[['Tipo','Nome', 'Lat', 'Lon','NUTS_LABEL', 'geometry']].sort_values(by='NUTS_LABEL').reset_index(drop=True)

#  NOTA: aqui, a referência tem que ser sempre 'right' para ir buscar as coordenadas dos pontos 
#  (e não as coordenadas dos poligonos dos NUTs), pois esta operação so guarda uma geometria

#criar variável com todos os 3 tipos de restaurantes
rest = pandas.concat([mcd, bk, tel], ignore_index=True).sort_values(by='NUTS_LABEL').reset_index(drop=True)


# agregar escolas e restaurantes por NUT para poder fazer choropleth
escolas_nut = geopandas.sjoin(portugal, escolas_crs, predicate='contains', how='left')[['Nome', 'NUTS_LABEL', 'geometry']].dissolve(by='NUTS_LABEL', aggfunc='count').reset_index().rename({'Nome': 'count'}, axis=1)
bk_nut = geopandas.sjoin(portugal, bk_crs, predicate='contains', how='left')[['Nome','NUTS_LABEL', 'geometry']].dissolve(by='NUTS_LABEL', aggfunc='count').reset_index().rename({'Nome': 'count'}, axis=1)
mcd_nut = geopandas.sjoin(portugal, mcd_crs, predicate='contains', how='left')[['Nome', 'NUTS_LABEL', 'geometry']].dissolve(by='NUTS_LABEL', aggfunc='count').reset_index().rename({'Nome': 'count'}, axis=1)
tel_nut = geopandas.sjoin(portugal, tel_crs, predicate='contains', how='left')[['Nome','NUTS_LABEL', 'geometry']].dissolve(by='NUTS_LABEL', aggfunc='count').reset_index().rename({'Nome': 'count'}, axis=1)
rest_nut = geopandas.sjoin(portugal, rest.drop(columns='NUTS_LABEL'), predicate='contains', how='left')[['Nome','NUTS_LABEL', 'geometry']].dissolve(by='NUTS_LABEL', aggfunc='count').reset_index().rename({'Nome': 'count'}, axis=1)

#  NOTA: aqui, a referência já tem que ser sempre 'left' para ir buscar as coordenadas dos polígonos dos NUTs

In [ ]:
# confirmar se há coordenadas iguais, o que há, porque escolas de ensinos diferentes e, como tal, com 
# nomes diferentes podem estar no mesmo edifício

#escolas[escolas['geometry'].duplicated(keep=False)] 

In [ ]:
#forma alternativa de converter df em gdf: 
#df['Coordinates']  = list(zip(df.Longitude, df.Latitude))
#df['Coordinates'] = df['Coordinates'].apply(Point)
#gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

In [ ]:
#grafico DEPOIS de mudar as coordenadas para UTM com unidades em metros
fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(15, 15))

ax.set_axis_off()
portugal.plot(ax=ax, edgecolor='k', alpha= 0.7, facecolor='white', figsize=(15,15))
escolas.plot(ax=ax, color='blue', alpha=0.5, label='Escolas')
bk.plot(ax=ax, color='red', alpha= 0.2, label ='Restaurantes')
mcd.plot(ax=ax, color='red', alpha=0.2, markersize=7)
tel.plot(ax=ax, color='red', alpha= 0.2, markersize=7)
ax.legend(fontsize=16,
         loc=(-0.4, 0.5),
         frameon=False)

scalebar = ScaleBar(dx=1, length_fraction=0.4, location='lower right', color='black',sep=7, pad=0, border_pad=0)
pyplot.gca().add_artist(scalebar)



#pp.title('urbanas', fontweight='bold')



#tirar os eixos mas manter um background cor diferente
#ax.set_facecolor((0, 0, 0, 0.05))
#ax.set_axis_off()
#ax.add_artist(ax.patch)
#ax.patch.set_zorder(-1)



#grafico1.set(xlim=(-100000, -80000), ylim=(-115000,-95000)) #lisboa 


#grafico1.set(xlim=(-150000, 170000), ylim=(-310000,285000))
#grafico1.set(xlim=(-130000, -55000), ylim=(-150000,-50000)) #lisboa 
#grafico1.set(xlim=(-60000, 0), ylim=(140000,180000)) #porto

#teste pontos
#grafico1.set(xlim=(-60000, 0), ylim=(90000,180000))
#escolas.loc[[0], 'geometry'].plot(ax=grafico1, color='blue', alpha=1, markersize=15)
#rest.loc[[1], 'geometry'].plot(ax=grafico1, color='black', alpha=1)

In [ ]:
fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(15, 15))

ax.set_axis_off()
portugal.plot(ax=ax, edgecolor='k', alpha= 0.7, facecolor='white', figsize=(15,15))
urbanas.plot(ax=ax, color='red', alpha=0.8)

#Not all handles can be turned into legend entries automatically, so it is often necessary to create an artist which can
#ver em https://matplotlib.org/tutorials/intermediate/legend_guide.html
red_patch = mpatches.Patch(color='red', label='Urbanas')


ax.legend(handles=[red_patch],
          fontsize=16,
         loc=(-0.4, 0.5),
         frameon=False)

scalebar = ScaleBar(dx=1, length_fraction=0.4, location='lower right', color='black',sep=7, pad=0, border_pad=0)
pyplot.gca().add_artist(scalebar)

In [ ]:
fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(15, 15))

ax.set_axis_off()
portugal.plot(ax=ax, edgecolor='k', alpha= 0.7, facecolor='white', figsize=(15,15))
vilas.plot(ax=ax, color='crimson', facecolor='none', label='Vilas')
ax.legend(fontsize=20,
         loc=(-0.4, 0.5),
         frameon=False)

scalebar = ScaleBar(dx=1, length_fraction=0.4, location='lower right', color='black',sep=7, pad=0, border_pad=0)
pyplot.gca().add_artist(scalebar)

In [ ]:
#distribuiçao nacional das escolas e de cada restaurante, por NUT
fig, (g2, g3, g4, g5, g6) = pyplot.subplots(nrows=1, ncols=5, figsize=(20, 16))
pyplot.tight_layout()

grafico2 = escolas_nut.plot(ax=g2, column='count', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
grafico3 = bk_nut.plot(ax=g3, column='count', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
grafico4 = mcd_nut.plot(ax=g4, column='count', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
grafico5 = tel_nut.plot(ax=g5, column='count', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
grafico6 = rest_nut.plot(ax=g6, column='count', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')

<h1></h1>
<h1>OBJECTIVO 1</h1>
<h2>Determinar, para cada escola em Portugal continental, qual a proximidade ao estabelecimento de fast-food (EFF) mais próximo</h2>

In [ ]:
#mudei os nomes das entradas para Capitalized
#acrescentei o tipo de rest nos csv dos mcd e tel
#padronizei os labels das colunas
#comparei visualmente no mapa para ver se batia certo


#retirado de https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe

def ckdnearest(gdA, gdB):
    nA = numpy.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = numpy.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pandas.concat(
        [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True),
         pandas.Series(dist, name='dist')], axis=1)
    gdf['dist'] = gdf['dist'] / 1000  #conversao de m para km
    return gdf

prox = ckdnearest(escolas.drop(columns=['Lat', 'Lon']).rename({'Nome':'ESCOLA'}, axis =1), rest.drop(columns=['Lat', 'Lon', 'NUTS_LABEL']))

In [ ]:
# agregar escolas e restaurantes por NUT para poder fazer choropleth
# criar as estatisticas por NUTs

prox_mean = geopandas.sjoin(portugal, prox.drop(columns=['NUTS_LABEL']), predicate='contains', how='left')[['NUTS_LABEL','geometry','ESCOLA','Nome','dist']].dissolve(by='NUTS_LABEL', aggfunc='mean').reset_index().rename({'dist': 'mean'}, axis=1)
prox_median = geopandas.sjoin(portugal, prox.drop(columns=['NUTS_LABEL']), predicate='contains', how='left')[['NUTS_LABEL','geometry','ESCOLA','Nome','dist']].dissolve(by='NUTS_LABEL', aggfunc='median').reset_index().rename({'dist': 'median'}, axis=1)
prox_min = geopandas.sjoin(portugal, prox.drop(columns=['NUTS_LABEL']), predicate='contains', how='left')[['NUTS_LABEL','geometry','dist']].dissolve(by='NUTS_LABEL', aggfunc='min').reset_index().rename({'dist': 'min'}, axis=1)
prox_max = geopandas.sjoin(portugal, prox.drop(columns=['NUTS_LABEL']), predicate='contains', how='left')[['NUTS_LABEL','geometry','dist']].dissolve(by='NUTS_LABEL', aggfunc='max').reset_index().rename({'dist': 'max'}, axis=1)

prox_nut = prox_mean.merge(prox_min.drop(columns='geometry'), on='NUTS_LABEL').merge(prox_median.drop(columns='geometry'), on='NUTS_LABEL').merge(prox_max.drop(columns='geometry'), on='NUTS_LABEL')

In [ ]:
prox_nut

In [ ]:
@ipywidgets.interact
def show_articles_more_than(column=['median', 'min'], x=(0, 35, 1)):
    return prox_nut.loc[prox_nut[column] < x]

In [ ]:
#fig, (g7, g8, g9) = pyplot.subplots(nrows=1, ncols=3, figsize=(20, 16))
#usamos a mediana pois ha outliers
pyplot.tight_layout()

grafico7 = prox_nut.plot(column='median', figsize=(20,16), legend=True, scheme='FisherJenks', cmap='Greens_r', edgecolor='k')
grafico7.set_axis_off()
#grafico8 = prox_nut.plot(ax=g8, column='min', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens_r', edgecolor='k')
#grafico9 = prox_nut.plot(ax=g9, column='max', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens_r', edgecolor='k')
#fig.canvas.toolbar_visible = False
#fig.canvas.header_visible = False
#fig.canvas.resizable = True

In [ ]:
seaborn.distplot(prox_nut['min'], rug=True)

<h1></h1>
<h1>OBJECTIVO 2</h1>
<h2>Determinar, para cada escola em Portugal continental, quantos EFF estão a curta distância (raios de 5 e de 10min a pé)</h2>
<h1>SEEMS TO BE WORKING UNTIL HERE</h1>


In [ ]:
# Para cada escola, criar um raio de 400m e de 800m e contar quantos rest estão nesse círculos
i = 0
raio = escolas[:]  # slice op para copiar o conteúdo e não linkar à variável antiga
raio['400m'] = ''  # criar novas duas colunas vazias
raio['800m'] = ''

for i in range(len(escolas)):
    raio.loc[i,'400m'] = len(rest[rest['geometry'].within(escolas['geometry'][i].buffer(400))])
    raio.loc[i,'800m'] = len(rest[rest['geometry'].within(escolas['geometry'][i].buffer(800))])

raio

In [ ]:
# agregar dados raio por NUT para poder fazer choropleth
# soma
raio_nut = geopandas.sjoin(portugal, raio.drop(columns=['NUTS_LABEL']), predicate='contains', how='left')[['NUTS_LABEL','geometry','Nome','400m','800m']].dissolve(by='NUTS_LABEL', aggfunc='sum').reset_index()


# média de restaurantes no raio especificado para as escolas de cada NUT
raio_mean = geopandas.sjoin(portugal, raio.drop(columns=['NUTS_LABEL']), predicate='contains', how='left')[['NUTS_LABEL','geometry','400m','800m']]
raio_mean[['400m', '800m']] = raio_mean[['400m', '800m']].apply(pandas.to_numeric) # por algum motivo que desconheço, estas colunas aparecem como objectos e impossibilitam a obtenção da média, pelo que tenho de converter em números
raio_mean = raio_mean.dissolve(by='NUTS_LABEL', aggfunc='mean').reset_index().rename({'400m': '400m_mean', '800m': '800m_mean'}, axis=1)

#juntar
raio_nut = raio_nut.merge(raio_mean.drop(columns=['geometry']), on='NUTS_LABEL')
raio_nut

In [ ]:
fig, (g12, g13) = pp.subplots(nrows=1, ncols=2, figsize=(20, 16))
#fig, (g10, g11, g12, g13) = pp.subplots(nrows=1, ncols=4, figsize=(20, 16))
pp.tight_layout()

#grafico10 = raio_nut.plot(ax=g10, column='400m', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
#grafico11 = raio_nut.plot(ax=g11, column='800m', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
grafico12 = raio_nut.plot(ax=g12, column='400m_mean', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')
grafico13 = raio_nut.plot(ax=g13, column='800m_mean', figsize=(5,5), legend=True, scheme='FisherJenks', cmap='Greens', edgecolor='k')

grafico12.set_axis_off()
grafico13.set_axis_off()

<h1></h1>
<h1>OBJECTIVO 3</h1>
<h2>Determinar se a localização dos EFF apresenta depêndencia espacial da localização das escolas (ou seja, se os EFFs exibem um padrão de clustering em redor das escolas)</h2>

- random mcd, bk e tel dentro dos buffers das vilas?
- ver se ha aleatoriedade ou se há tendendecia de clustering em volta das escolas estatisticamente significativa

Kcross inohomgenous nao executval no python? Tive que exportar para R
extraí as "coordenadas" da geometria para celulas à parte
exportei para shapefile
converti em ppp no R (ver https://github.com/jlevente/publications/blob/master/cross-k/calc_crossk.R)

fiz a Kcross inhomegenous com envelope:
    raio 1500m
    lambdaX = vilas_ppm (para ter em consideração a maior probabilidade de calhar junto a vilas)
    nrank=1 (para definir como min e max do envelope o n-esimo valor mínimo e o nésimo valor maximo
    global = TRUE (para homegenizar as curvas e dar um uma probabilidade?)
    correction='translation' (? resulta!)

<s>Fiz exactamente o mesmo, mas com Lcross inhomegenous, pois esta permite ter um gráfico mais facilmente interpretavel (pois tem menor variação com o r?)
Acrescentei na expressao do gráfico ".-r ~ r", para que a recta da H0 fosse horizontal (e não diagonal)</s>

Queria acrescentar as urbanas à heterogeneidade, mas sendo polígonos torna-se complicado.
Nao consigo misturar poligonos com pontos, pois ha infinitos pontos dentro de um poligono e vou distorcer a intensidade

Como tal, vou criar 2 analises de K functions: 
    uma fora das zonas urbanas, window = portugal - urbanas. tenho de retirar todos os pontos dentro das areas urbanas e so considerar os outros (atencao à aresta!!)
    a outra dentro das areas urbanas, em que window = urbanas


ver se faz diferença usar na intesnidade (lambdaX) o ppm ou a density

falta saber como fazer cloropeth disto


In [ ]:
# Colocar na mesma dataframe as escolas e os restaurantes para poder passar a ppp. 
# Note-se que é necessário fazer a distinção entre os dois tipos de ponto de cada dataframe resultante, pelo que
# é necessário acrescentar o tipo para poder fazer a distincao dos pontos escolas vs restaurante no R na funcao ppp

temp_escolas = escolas.drop(columns=['Lat', 'Lon'])
temp_escolas.insert(loc=0, column='Tipo', value='escola')

# bk, mcd, tel
uniao_escolas_bk = pandas.concat([bk.drop(columns=['Lat', 'Lon']), temp_escolas], ignore_index=True).sort_values(by='Tipo').reset_index(drop=True)
uniao_escolas_mcd = pandas.concat([mcd.drop(columns=['Lat', 'Lon']), temp_escolas], ignore_index=True).sort_values(by='Tipo').reset_index(drop=True)
uniao_escolas_tel = pandas.concat([tel.drop(columns=['Lat', 'Lon']), temp_escolas], ignore_index=True).sort_values(by='Tipo').reset_index(drop=True)

# para os restaurantes no geral
# Note-se que aqui é preciso acrescentar o tipo da variável restaurante, para distinguir das escolas, e aqui
# já não importa a distinção entre diferentes tipos de restaurante

temp_rest = rest.drop(columns=['Tipo','Lat', 'Lon'])
temp_rest.insert(loc=0, column ='Tipo', value='restaurante')
uniao_escolas_rest = pandas.concat([temp_rest, temp_escolas], ignore_index=True).sort_values(by='Tipo').reset_index(drop=True)

In [ ]:
uniao_escolas_rest

In [ ]:
#verificar que tantos os restaurantes, como as escolas como as vilas ou estão dentro ou fora das areas urbanas, 
#de forma a não perder os que poderiam estar nas bordas

for a in (vilas, uniao_escolas_bk, uniao_escolas_mcd, uniao_escolas_tel, uniao_escolas_rest):
    print(len(a[a.within(urbanas_poligono)]) + len(a[a.disjoint(urbanas_poligono)]) == len(a))

In [ ]:
#separar dentro das areas urbanas vs fora
uniao_escolas_rest_urbanas = uniao_escolas_rest[uniao_escolas_rest.within(urbanas_poligono)]
uniao_escolas_rest_rural = uniao_escolas_rest[uniao_escolas_rest.disjoint(urbanas_poligono)]

vilas_urbanas = vilas[vilas.within(urbanas_poligono)]
vilas_rural = vilas[vilas.disjoint(urbanas_poligono)]

In [ ]:
# Extração das coordenadas x e y da geometria dos pontos (note-se que estão em CRS diferente das lat e lon originais!)

def extrair_xy(lista):
    x = [x for x,y in zip(lista['geometry'].x, lista['geometry'].y)]
    y = [y for x,y in zip(lista['geometry'].x, lista['geometry'].y)]
    lista.insert(loc=len(lista.columns), column='x', value=x)
    lista.insert(loc=len(lista.columns), column='y', value=y)
    
extrair_xy(uniao_escolas_bk)
extrair_xy(uniao_escolas_mcd)
extrair_xy(uniao_escolas_tel)

extrair_xy(vilas_urbanas)
extrair_xy(vilas_rural)
extrair_xy(uniao_escolas_rest_urbanas)
extrair_xy(uniao_escolas_rest_rural)

In [ ]:
#pt_sem_urbanas
rural = portugal[:][['NUTS_LABEL','geometry']]
rural['geometry'] = portugal.difference(urbanas_poligono)

In [ ]:
# exportar para shapefile a ser lida n R
portugal[['NUTS_LABEL', 'geometry']].to_file('shapefiles/portugal.shp', driver='ESRI Shapefile')
rural.to_file('shapefiles/rural.shp', driver='ESRI Shapefile')
urbanas.to_file('shapefiles/urbanas.shp', driver='ESRI Shapefile')

vilas_urbanas.to_file('shapefiles/vilas_urbanas.shp', driver='ESRI Shapefile')
vilas_rural.to_file('shapefiles/vilas_rural.shp', driver='ESRI Shapefile')

uniao_escolas_bk.to_file('shapefiles/bk.shp', driver='ESRI Shapefile')
uniao_escolas_mcd.to_file('shapefiles/mcd.shp', driver='ESRI Shapefile')
uniao_escolas_tel.to_file('shapefiles/tel.shp', driver='ESRI Shapefile')
uniao_escolas_rest.to_file('shapefiles/rest.shp', driver='ESRI Shapefile')
uniao_escolas_rest_rural.to_file('shapefiles/rest_rural.shp', driver='ESRI Shapefile')
uniao_escolas_rest_urbanas.to_file('shapefiles/rest_urbanas.shp', driver='ESRI Shapefile')